In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
os.environ["CUDA_LAUNCH_BLOCKING"]='1'

In [2]:
#import torch
from mamba_ssm.modules.mamba_simple import Mamba

In [3]:
block = Mamba(d_model=2560).to("cuda")

In [4]:
print(block)

Mamba(
  (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
  (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
  (act): SiLU()
  (x_proj): Linear(in_features=5120, out_features=192, bias=False)
  (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
  (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
)


In [5]:
import torch

In [6]:
# each mamba block is of input & output feature dimension 2560
B, S, D = 1, 32, 2560  # Example: Batch=1, Sequence Length=32, Features=2560
input_tensor = torch.randn(B, S, D).to("cuda")

In [7]:
# inference of block takes longer than whole model? (at first)

In [8]:
block(input_tensor).shape

input to original 'compute_attn_matrix'
delta: (<class 'torch.Tensor'>, torch.Size([1, 5120, 32]))
delta_bias: (<class 'torch.nn.parameter.Parameter'>, torch.Size([5120]))
A: (<class 'torch.Tensor'>, torch.Size([5120, 16]))
B: (<class 'torch.Tensor'>, torch.Size([1, 1, 16, 32]))
C: (<class 'torch.Tensor'>, torch.Size([1, 1, 16, 32]))
L: (<class 'int'>, 32)
x.shape: (<class 'torch.Tensor'>, torch.Size([1, 5120, 32]))
dt: (<class 'torch.Tensor'>, torch.Size([1, 5120, 32]))
in compute attn matrix: dt, torch.Size([1, 5120, 32]); dt_bias, torch.Size([1, 5120, 1])
dt: torch.Size([1, 5120, 32])


KeyboardInterrupt: 

In [1]:
conv_dim = 77
conv_bias = True
d_conv = 4
#factory_kwargs = {"device": "cuda", "dtype": "int32"}

In [4]:
import torch.nn as nn
conv1d = nn.Conv1d(
            in_channels=conv_dim,
            out_channels=conv_dim,
            bias=conv_bias,
            kernel_size=d_conv,
            groups=conv_dim,
            padding=d_conv - 1,
            #**factory_kwargs,
        )

In [5]:
conv1d.bias.shape

torch.Size([77])

In [9]:
# test segsum
import torch
from mamba_ssm.modules.ssd_minimal import segsum
a = torch.rand((32))
torch.exp(segsum(a))

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.5322e+00, 1.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.7564e+00, 1.1463e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [5.1801e+06, 3.3809e+06, 2.9493e+06,  ..., 1.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.2382e+07, 8.0815e+06, 7.0499e+06,  ..., 2.3903e+00, 1.0000e+00,
         0.0000e+00],
        [2.6271e+07, 1.7146e+07, 1.4957e+07,  ..., 5.0715e+00, 2.1217e+00,
         1.0000e+00]])

In [8]:
segsum(a).shape

torch.Size([80, 32, 32])